In [1]:
from cuttings import *

import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import time
import copy
import PIL.Image as Image
import random

import torch.optim as optim
import torchvision.transforms as tf
from torch.utils import data 
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

import pickle 

In [2]:
# torchvision for pre-trained models
from torchvision import models

#### Classes

In [3]:
class MinMaxNormalization(object):
    """
    Normalized (Min-Max) the image.
    """
    def __init__(self, vmin=0, vmax=1):
        """
        Constructor of the grayscale transform.
        ----------
        INPUT
            |---- vmin (float / int) the desired minimum value.
            |---- vmax (float / int) the desired maximum value.
        OUTPUT
            |---- None
        """
        self.vmin = vmin
        self.vmax = vmax

    def __call__(self, image, mask=None):
        """
        Apply a Min-Max Normalization to the image.
        ----------
        INPUT
            |---- image (PIL.Image) the image to normalize.
        OUTPUT
            |---- image (np.array) the normalized image.
        """
        arr = np.array(image).astype('float32')
        arr = (arr - arr.min()) / (arr.max() - arr.min())
        arr = (self.vmax - self.vmin) * arr + self.vmin
        return arr

In [4]:
class Cuttings_Dataset(data.Dataset):
    def __init__(self, sample_df, data_path, mean, std, resize,data_augmentation=True):
        """
        Constructor of the dataset.
        """
        data.Dataset.__init__(self)
        self.sample_df = sample_df
        self.data_path = data_path
        self.data_augmentation = data_augmentation
        
        self.transform =  tf.Compose([tf.Grayscale(num_output_channels=3),
                                        MinMaxNormalization(),
                                        tf.ToTensor(),
                                        tf.Normalize((mean,mean,mean),(std,std,std))])
        
        if data_augmentation:
            self.transform = tf.Compose([tf.Grayscale(num_output_channels=3),
                                            tf.RandomVerticalFlip(p=0.5),
                                            tf.RandomHorizontalFlip(p=0.5),
                                            tf.RandomRotation([-90,90],resample=False, expand=False, center=None, fill=None),
                                            MinMaxNormalization(),
                                            tf.ToTensor(),
                                            tf.Normalize((mean,mean,mean),(std,std,std))])
        
        if resize:
            self.transform = tf.Compose([tf.Grayscale(num_output_channels=3),
                                            tf.Resize(224),
                                            tf.RandomVerticalFlip(p=0.5),
                                            tf.RandomHorizontalFlip(p=0.5),
                                            tf.RandomRotation([-90,90],resample=False, expand=False, center=None, fill=None),
                                            MinMaxNormalization(),
                                            tf.ToTensor(),
                                            tf.Normalize((mean,mean,mean),(std,std,std))])
    def __len__(self):
        """
        Get the number of samples in the dataset.
        """
        return self.sample_df.shape[0]


    def __getitem__(self, idx):
        """
        Get an item from the dataset.
        """
        # load image
        im = Image.open(self.data_path + self.sample_df.loc[idx,'path'])
        # load label
        label = torch.tensor(self.sample_df.loc[idx,'rock_type'])
        
        im = self.transform(im)
        return im,label

#### Loading data

In [5]:
classes = np.array(['ML', 'MS','BL','GN','OL'])

date = '05_05_20'

df = pd.read_csv('train/csv_'+date+'/train'+date+'_final.csv',index_col=0)

train,val = train_test_split(df,test_size=0.2,random_state=0,stratify=df['rock_type'])
cuttings_datasets = {}

cuttings_datasets['train'] = Cuttings_Dataset(train.reset_index(drop=True),'',data_augmentation=True)
cuttings_datasets['val'] = Cuttings_Dataset(val.reset_index(drop=True),'',data_augmentation=False)
cuttings_datasets['test'] = Cuttings_Dataset(pd.read_csv('test'+'/csv_'+date+'/'+'test'+date+'_final.csv',index_col=0),'',data_augmentation=False)

dataloaders = {x: torch.utils.data.DataLoader(cuttings_datasets[x], batch_size=16,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'val', 'test']}

dataset_sizes = {x: len(cuttings_datasets[x]) for x in ['train','val','test']}

In [6]:
# Flag for feature extracting. When False, we finetune the whole model,
# when True we only update the reshaped layer params

In [7]:
def initialize_model(num_classes, feature_extract=True, use_pretrained=True):
    
    # By default, when we load a pretrained model all of the parameters have .requires_grad=True
    model_ft = models.resnet18(pretrained=use_pretrained)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, num_classes)
    return model_ft

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [8]:
model_ft = initialize_model(len(classes))

In [9]:
for name,param in model_ft.named_parameters():
    if param.requires_grad == True:
        print("\t",name)

	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer3.0.conv1.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv2.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.downsample.0.weight
	 layer3.0.downsample.1.weight
	 layer3.0.downsample.1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.conv2.weight
	 layer3.1.bn2.weight
	 layer

In [10]:
def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    loss_train = []
    acc_train = []
    loss_val = []
    acc_val = []
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        running_loss_train = 0.0
        running_corrects_train = 0
            
        model.train()# Set model to training mode
        
        # Iterate over data Training
        for inputs, labels in dataloaders['train']:
            inputs = inputs.to(device)
            inputs.require_grad = True
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            loss = criterion(outputs, labels)

            # backward + optimize
            loss.backward()
            optimizer.step()

            # statistics
            running_loss_train += loss.item() * inputs.size(0)
            running_corrects_train += torch.sum(preds == labels.data)
            
        epoch_loss_train = running_loss_train / dataset_sizes["train"]
        epoch_acc_train = running_corrects_train.double() / dataset_sizes["train"]
        loss_train.append(epoch_loss_train)
        acc_train.append(epoch_acc_train)
        
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                "Train", epoch_loss_train, epoch_acc_train))
        
        running_loss_val = 0.0
        running_corrects_val = 0
            
        model.eval() # Set model to evaluate mode
        
        # Iterate over data Evaluation
        for inputs, labels in dataloaders["val"]:
            inputs = inputs.to(device)
            labels = labels.to(device)
                
            with torch.no_grad():
                # forward
                outputs = model(inputs)
                preds = torch.argmax(outputs, 1)
                loss = criterion(outputs, labels)   

            # statistics
            running_loss_val += loss.item() * inputs.size(0)
            running_corrects_val += torch.sum(preds == labels.data)

        epoch_loss_val = running_loss_val / dataset_sizes["val"]
        epoch_acc_val = running_corrects_val.double() / dataset_sizes["val"]
        loss_val.append(epoch_loss_val)
        acc_val.append(epoch_acc_val)
        
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                "Val", epoch_loss_val, epoch_acc_val))
            
        # deep copy the model
        if epoch_acc_val > best_acc:
            best_acc = epoch_acc_val
            best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, loss_train, acc_train, loss_val, acc_val

In [11]:
# prediciton
def prediciton(model):
    preds_vec = []
    true_vec = []
    
    model.eval()
    with torch.no_grad():
        for inputs, labels in dataloaders['test']:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            preds_vec+=preds.tolist()
            true_vec+=labels.tolist()
    return preds_vec, true_vec

In [12]:
# Get Cuda
date= '13_06_20'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# Train Model
list_loss_train = []
list_acc_train = []
list_loss_val = []
list_acc_val = []
list_preds_vec = []
list_true_vec = []

for i in range(3):
    model_ft = initialize_model(len(classes))

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model_ft.parameters())

    model_ft = model_ft.to(device)

    model_ft, loss_train, acc_train, loss_val, acc_val = train_model(model_ft, criterion, optimizer,
                       num_epochs=200)
    
    preds_vec, true_vec = prediciton(model_ft)
    
    list_loss_train.append(loss_train)
    list_acc_train.append(acc_train)
    list_loss_val.append(loss_val)
    list_acc_val.append(acc_val)
    list_preds_vec.append(preds_vec)
    list_true_vec.append(true_vec)
    
    torch.save(model_ft, f'results/Resnet18_{i}_{date}')
# Save Pickles
results = open(f'results/Resnet18_results_{date}', 'wb')
pickle.dump([[list_loss_train],[list_acc_train],[list_loss_val],[list_acc_val],[list_preds_vec],[list_true_vec]],results)
results.close()

cuda:0
Epoch 0/199
----------
Train Loss: 0.7563 Acc: 0.7275
Val Loss: 0.5986 Acc: 0.7920

Epoch 1/199
----------
Train Loss: 0.5785 Acc: 0.7900
Val Loss: 1.3576 Acc: 0.6170

Epoch 2/199
----------
Train Loss: 0.5094 Acc: 0.8230
Val Loss: 0.3969 Acc: 0.8500

Epoch 3/199
----------
Train Loss: 0.4280 Acc: 0.8473
Val Loss: 0.4606 Acc: 0.8240

Epoch 4/199
----------
Train Loss: 0.3735 Acc: 0.8700
Val Loss: 0.4108 Acc: 0.8530

Epoch 5/199
----------
Train Loss: 0.3902 Acc: 0.8593
Val Loss: 0.3459 Acc: 0.8890

Epoch 6/199
----------
Train Loss: 0.3430 Acc: 0.8668
Val Loss: 0.3694 Acc: 0.8490

Epoch 7/199
----------
Train Loss: 0.3452 Acc: 0.8688
Val Loss: 0.3377 Acc: 0.8650

Epoch 8/199
----------
Train Loss: 0.3396 Acc: 0.8765
Val Loss: 0.2780 Acc: 0.8850

Epoch 9/199
----------
Train Loss: 0.3022 Acc: 0.8912
Val Loss: 0.2681 Acc: 0.9000

Epoch 10/199
----------
Train Loss: 0.2866 Acc: 0.8972
Val Loss: 0.2672 Acc: 0.8920

Epoch 11/199
----------
Train Loss: 0.2960 Acc: 0.8960
Val Loss: 0.2

Val Loss: 0.1679 Acc: 0.9490

Epoch 97/199
----------
Train Loss: 0.0819 Acc: 0.9688
Val Loss: 0.2151 Acc: 0.9300

Epoch 98/199
----------
Train Loss: 0.0796 Acc: 0.9725
Val Loss: 0.1820 Acc: 0.9310

Epoch 99/199
----------
Train Loss: 0.0754 Acc: 0.9713
Val Loss: 0.1540 Acc: 0.9540

Epoch 100/199
----------
Train Loss: 0.0681 Acc: 0.9753
Val Loss: 0.2046 Acc: 0.9330

Epoch 101/199
----------
Train Loss: 0.0677 Acc: 0.9735
Val Loss: 0.1245 Acc: 0.9570

Epoch 102/199
----------
Train Loss: 0.0552 Acc: 0.9800
Val Loss: 0.1814 Acc: 0.9470

Epoch 103/199
----------
Train Loss: 0.0822 Acc: 0.9713
Val Loss: 0.1577 Acc: 0.9440

Epoch 104/199
----------
Train Loss: 0.0967 Acc: 0.9690
Val Loss: 0.1348 Acc: 0.9490

Epoch 105/199
----------
Train Loss: 0.0799 Acc: 0.9718
Val Loss: 0.1962 Acc: 0.9210

Epoch 106/199
----------
Train Loss: 0.0942 Acc: 0.9710
Val Loss: 0.1704 Acc: 0.9400

Epoch 107/199
----------
Train Loss: 0.0750 Acc: 0.9748
Val Loss: 0.1287 Acc: 0.9440

Epoch 108/199
----------
Tr

Train Loss: 0.0281 Acc: 0.9895
Val Loss: 0.1533 Acc: 0.9600

Epoch 193/199
----------
Train Loss: 0.0193 Acc: 0.9930
Val Loss: 0.1661 Acc: 0.9600

Epoch 194/199
----------
Train Loss: 0.0331 Acc: 0.9885
Val Loss: 0.1498 Acc: 0.9580

Epoch 195/199
----------
Train Loss: 0.0460 Acc: 0.9855
Val Loss: 0.1338 Acc: 0.9630

Epoch 196/199
----------
Train Loss: 0.0256 Acc: 0.9920
Val Loss: 0.1465 Acc: 0.9620

Epoch 197/199
----------
Train Loss: 0.0221 Acc: 0.9928
Val Loss: 0.1762 Acc: 0.9540

Epoch 198/199
----------
Train Loss: 0.0264 Acc: 0.9898
Val Loss: 0.1958 Acc: 0.9550

Epoch 199/199
----------
Train Loss: 0.0349 Acc: 0.9880
Val Loss: 0.1963 Acc: 0.9500

Training complete in 208m 26s
Best val Acc: 0.964000
Epoch 0/199
----------
Train Loss: 0.8043 Acc: 0.7105
Val Loss: 0.5178 Acc: 0.8100

Epoch 1/199
----------
Train Loss: 0.5435 Acc: 0.8040
Val Loss: 0.9197 Acc: 0.6310

Epoch 2/199
----------
Train Loss: 0.5132 Acc: 0.8200
Val Loss: 0.4394 Acc: 0.8230

Epoch 3/199
----------
Train Los

Train Loss: 0.0759 Acc: 0.9720
Val Loss: 0.1504 Acc: 0.9540

Epoch 89/199
----------
Train Loss: 0.0906 Acc: 0.9667
Val Loss: 0.1492 Acc: 0.9540

Epoch 90/199
----------
Train Loss: 0.0849 Acc: 0.9673
Val Loss: 0.2002 Acc: 0.9370

Epoch 91/199
----------
Train Loss: 0.0995 Acc: 0.9640
Val Loss: 0.1494 Acc: 0.9510

Epoch 92/199
----------
Train Loss: 0.0743 Acc: 0.9733
Val Loss: 0.3126 Acc: 0.8940

Epoch 93/199
----------
Train Loss: 0.0789 Acc: 0.9723
Val Loss: 0.1902 Acc: 0.9430

Epoch 94/199
----------
Train Loss: 0.0710 Acc: 0.9740
Val Loss: 0.1497 Acc: 0.9440

Epoch 95/199
----------
Train Loss: 0.0684 Acc: 0.9755
Val Loss: 0.1714 Acc: 0.9420

Epoch 96/199
----------
Train Loss: 0.0779 Acc: 0.9738
Val Loss: 0.1275 Acc: 0.9530

Epoch 97/199
----------
Train Loss: 0.0730 Acc: 0.9745
Val Loss: 0.1944 Acc: 0.9440

Epoch 98/199
----------
Train Loss: 0.0704 Acc: 0.9758
Val Loss: 0.1398 Acc: 0.9560

Epoch 99/199
----------
Train Loss: 0.0705 Acc: 0.9725
Val Loss: 0.1608 Acc: 0.9540

Epoc

Train Loss: 0.0283 Acc: 0.9913
Val Loss: 0.1860 Acc: 0.9470

Epoch 185/199
----------
Train Loss: 0.0292 Acc: 0.9908
Val Loss: 0.1806 Acc: 0.9560

Epoch 186/199
----------
Train Loss: 0.0324 Acc: 0.9893
Val Loss: 0.1632 Acc: 0.9550

Epoch 187/199
----------
Train Loss: 0.0319 Acc: 0.9880
Val Loss: 0.2361 Acc: 0.9440

Epoch 188/199
----------
Train Loss: 0.0349 Acc: 0.9873
Val Loss: 0.1909 Acc: 0.9520

Epoch 189/199
----------
Train Loss: 0.0262 Acc: 0.9913
Val Loss: 0.1908 Acc: 0.9480

Epoch 190/199
----------
Train Loss: 0.0240 Acc: 0.9908
Val Loss: 0.1595 Acc: 0.9620

Epoch 191/199
----------
Train Loss: 0.0300 Acc: 0.9875
Val Loss: 0.3150 Acc: 0.9450

Epoch 192/199
----------
Train Loss: 0.0342 Acc: 0.9883
Val Loss: 0.3653 Acc: 0.9350

Epoch 193/199
----------
Train Loss: 0.0468 Acc: 0.9873
Val Loss: 0.2515 Acc: 0.9310

Epoch 194/199
----------
Train Loss: 0.0873 Acc: 0.9765
Val Loss: 0.1623 Acc: 0.9500

Epoch 195/199
----------
Train Loss: 0.0390 Acc: 0.9880
Val Loss: 0.1422 Acc: 0

Train Loss: 0.0873 Acc: 0.9673
Val Loss: 0.1502 Acc: 0.9500

Epoch 81/199
----------
Train Loss: 0.0812 Acc: 0.9693
Val Loss: 0.1528 Acc: 0.9570

Epoch 82/199
----------
Train Loss: 0.0837 Acc: 0.9675
Val Loss: 0.2177 Acc: 0.9160

Epoch 83/199
----------
Train Loss: 0.0839 Acc: 0.9683
Val Loss: 0.1517 Acc: 0.9530

Epoch 84/199
----------
Train Loss: 0.0859 Acc: 0.9677
Val Loss: 0.3455 Acc: 0.8910

Epoch 85/199
----------
Train Loss: 0.1073 Acc: 0.9645
Val Loss: 0.1481 Acc: 0.9440

Epoch 86/199
----------
Train Loss: 0.0890 Acc: 0.9683
Val Loss: 0.1614 Acc: 0.9420

Epoch 87/199
----------
Train Loss: 0.0844 Acc: 0.9665
Val Loss: 0.1498 Acc: 0.9490

Epoch 88/199
----------
Train Loss: 0.0922 Acc: 0.9673
Val Loss: 0.1936 Acc: 0.9360

Epoch 89/199
----------
Train Loss: 0.0851 Acc: 0.9705
Val Loss: 0.1809 Acc: 0.9500

Epoch 90/199
----------
Train Loss: 0.0711 Acc: 0.9730
Val Loss: 0.1768 Acc: 0.9410

Epoch 91/199
----------
Train Loss: 0.0810 Acc: 0.9685
Val Loss: 0.1891 Acc: 0.9400

Epoc

Train Loss: 0.0398 Acc: 0.9885
Val Loss: 0.1977 Acc: 0.9500

Epoch 177/199
----------
Train Loss: 0.0440 Acc: 0.9830
Val Loss: 0.1543 Acc: 0.9600

Epoch 178/199
----------
Train Loss: 0.0357 Acc: 0.9888
Val Loss: 0.1582 Acc: 0.9570

Epoch 179/199
----------
Train Loss: 0.0270 Acc: 0.9893
Val Loss: 0.1976 Acc: 0.9510

Epoch 180/199
----------
Train Loss: 0.0264 Acc: 0.9915
Val Loss: 0.2016 Acc: 0.9580

Epoch 181/199
----------
Train Loss: 0.0313 Acc: 0.9888
Val Loss: 0.1783 Acc: 0.9540

Epoch 182/199
----------
Train Loss: 0.0442 Acc: 0.9858
Val Loss: 0.2314 Acc: 0.9440

Epoch 183/199
----------
Train Loss: 0.0461 Acc: 0.9825
Val Loss: 0.1502 Acc: 0.9570

Epoch 184/199
----------
Train Loss: 0.0177 Acc: 0.9942
Val Loss: 0.1522 Acc: 0.9630

Epoch 185/199
----------
Train Loss: 0.0401 Acc: 0.9870
Val Loss: 0.1503 Acc: 0.9590

Epoch 186/199
----------
Train Loss: 0.0414 Acc: 0.9855
Val Loss: 0.1551 Acc: 0.9600

Epoch 187/199
----------
Train Loss: 0.0289 Acc: 0.9888
Val Loss: 0.1683 Acc: 0